In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from pprint import pprint

# 판매량을 예측할수 있을까?

### 판매량에 영향을 끼칠수 있는 변수는 무었이 있을까?
+ 가수 정보
    + 대중 관심도 
        + 검색량 : 네이버, 구글, 유튜브 검색량
        + 선호도 : 댓글, 트위터 분석
    + 팬
        + 수 : 멜론 팬
        + 성별 : 비율
        + 연령 : 비율
    + 소속사
+ 노래 정보
    + 대중 관심도
        + 검색량 : 네이버, 구글, 유튜브 검색량
        + 선호도 : 댓글, 트위터 분석, 멜론 좋아요수
    + 타이틀
    + 장르
+ 대중매체 노출도
    + ???

### 가수들이 너무 많음
+ 노래를 제한하자

# 가온차트 & 멜론 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

response_download = requests.get("http://www.gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=S1020&targetTime=02&hitYear=2016&termGbn=week" ) 
response_social =   requests.get("http://www.gaonchart.co.kr/main/section/chart/social.gaon?nationGbn=S&serviceGbn=&targetTime=02&hitYear=2016&termGbn=week" ) 

soup_download = BeautifulSoup(response_download.content.decode('utf-8'), 'html.parser')
soup_social = BeautifulSoup(response_social.content.decode('utf-8'), 'html.parser')

titles_download = soup_download.select('tr p')
titles_social = soup_social.select('tr p')
counts = soup_social.select('td.count')

In [ ]:
download = [titles_download[i].text for i in range(500)]
social = [titles_social[i].text for i in range(400)]
count = [counts[i].text for i in range(100)]

In [ ]:
columns_download = ['Title', 'Artist/album', 'Download', 'Production', 'Distribution']
df_download = pd.DataFrame(np.array(download).reshape((100,5)), index=range(1,100+1), columns = columns_download)
df_download.insert(loc=list(df_download).index('Title')+1 ,column='Artist', value=df_download.loc[:,'Artist/album'].apply(lambda x: x.split('|')[0]))
df_download.insert(loc=list(df_download).index('Artist')+1 ,column='album', value=df_download.loc[:,'Artist/album'].apply(lambda x: x.split('|')[1]))
df_download.__delitem__('Artist/album')

In [ ]:
columns_social = ['Title', 'Artist', 'Production', 'Distribution']
df_social = pd.DataFrame(np.array(social).reshape((100,4)), index=range(1,100+1), columns = columns_social)
df_social.loc[:, 'Artist'] = df_social.loc[:, 'Artist'].apply(lambda x: x.split('|')[0])
df_social.insert(loc=list(df_social).index('Artist')+1 ,column='count', value=count)
df_social.__delitem__('Distribution')

In [ ]:
df_download.loc[:, 'Artist'] = df_download.loc[:, 'Artist'].apply(lambda x: re.sub('★', '', x))
df_download.loc[:, 'Artist'] = df_download.loc[:, 'Artist'].apply(lambda x: re.sub(' ?\(.+?\)', '', x)).apply(lambda x: re.sub(' X ', ', ', x)).apply(lambda x: x.split(', '))

In [ ]:
df_download

In [ ]:
import itertools
singer_list = list(set(itertools.chain(*df_download.loc[:, 'Artist'])))
singer_list[singer_list.index('f')] = 'f(x)'

In [ ]:
from selenium import webdriver

chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver)

singer_fan = {}

for singer_el in singer_list:
    artist_url = "http://www.melon.com/search/artist/index.htm?q={}&section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&ipath=srch_form"
    driver.get(artist_url.format(singer_el))
    fan_count = driver.find_element_by_css_selector('div.atist_info > dl > dd > span.cnt_fan > span')
    
    singer_fan[singer_el] =  int(re.sub(',', '', fan_count.text))
driver.quit()

In [ ]:
singer_list

In [ ]:
singer_fan['f(x)']

In [ ]:
Artist_fan = df_download.loc[:, 'Artist'].apply(lambda x: sum([singer_fan[a] if a=='f' for a in x]))

In [ ]:
df_download.insert(loc=list(df_download).index('Artist')+1 ,column='Artist_fan', value=Artist_fan)

# 네이버 데이터랩

In [ ]:
#-*- coding: utf-8 -*-
import os
import sys
import urllib.request
client_id = 'bHdSL0QqLlouTVx4ruLr'
client_secret = '7ZqBvDGPWl'
url = "https://openapi.naver.com/v1/datalab/search"
body = "{\"startDate\":\"2016-01-01\",\"endDate\":\"2017-11-30\",\"timeUnit\":\"week\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

# 주제 변경!

### 클러스터링을 해보자

+ 가수, 노래, 앨범

### 앨범 불러오기

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
# 2 ~ 53, 1~ 48

album_df = pd.DataFrame(columns=['album'])
for i in range(1, 48+1):
    response = requests.get("http://www.gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=S1020&targetTime={}&hitYear=2017&termGbn=week".format(i)) 
    soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
    album = soup.select('tr p.singer')
    album_df_temp = pd.DataFrame({'singer': [j.text.split('|')[0] for j in album], 'album' : [j.text.split('|')[1] for j in album]})
    album_df = pd.concat([album_df, album_df_temp])

In [3]:
album_df = album_df.drop_duplicates(['album', 'singer'])

In [4]:
import itertools
album_list = list(set(itertools.chain(album_df.loc[:,'album'])))
album_list = list(set(itertools.chain(*[i.split(', ') for i in album_list])))

In [5]:
album_df1 = pd.DataFrame()
album_df1['album'] = album_df.loc[:, 'album'].apply(lambda x: re.sub(' ?\(.+?\)', '', x))
album_df1['singer'] = album_df.loc[:, 'singer'].apply(lambda x: re.sub(' ?\(.+?\)', '', x)).apply(lambda x: re.sub(' X ', ', ', x)).apply(lambda x: x.split(', '))

### 앨범의 번호를 긁어오자

In [6]:
from bs4 import BeautifulSoup

naver_list = []
for i in range(len(album_df1)):
    url = 'http://music.naver.com/search/search.nhn?query={} {}&x=0&y=0'.\
           format(album_df1.iloc[i,:][0], album_df1.iloc[i,:][1][0])
    response = requests.get(url) 
    soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

    album_num = soup.select('table > tbody > tr > td.album > a')
    if len(album_num) != 0:
        naver_list.append(re.search(r'i:([0-9]*)', str(album_num[0])).group(1))
    else:
        naver_list.append(None)

In [7]:
album_df['naver'] = naver_list

In [43]:
melon_list = []
for i in range(len(album_df1)):
    url = 'http://www.melon.com/search/total/index.htm?q={} {}&section=&searchGnbYn=Y&kkoSpl=N&kkoDpType=&ipath=srch_form'.\
           format(album_df1.iloc[i,:][0], album_df1.iloc[i,:][1][0])
    response = requests.get(url) 
    soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

    album_num = soup.select('tbody > tr > td > div.wrap > div.ellipsis > a.fc_mgray')

    if len(album_num) != 0:
        melon_list.append(re.search(r'goAlbumDetail\(\'([0-9]*)\'', str(album_num)).group(1))
    else:
        melon_list.append(None)

In [ ]:
# album_df.__delitem__('melon')

In [ ]:
album_df.insert(loc=list(album_df).index('naver')+1 ,column='melon', value=melon_list)

In [12]:
from selenium import webdriver
import time

chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver)

Mnet_list = []
for i in range(len(album_df1)):
    url = 'http://search.mnet.com/search/index.asp?q={} {}'.format(album_df1.iloc[i,:][0], album_df1.iloc[i,:][1][0])
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    album_num = soup.select('tbody > tr > td.MMLItemAlbum > div > a')[0]
    if len(album_num) != 0:
        Mnet_list.append(re.search(r'http://www.mnet.com/album/([0-9]*)', str(album_num)).group(1))
    else:
        Mnet_list.append(None)
        print(i)
driver.quit()

In [13]:
album_df.insert(loc=list(album_df).index('melon')+1 ,column='Mnet', value=Mnet_list)

In [51]:
album_df4 = album_df.drop_duplicates(['album'])

In [52]:
album_df5 = album_df4.loc[:, ['album', 'naver', 'melon', 'Mnet']].dropna(axis=0, how='any').copy()

In [53]:
album_df5.set_index(np.arange(len(album_df5)), inplace=True)

In [55]:
album_df5.to_csv('album_df5.csv', index=False, encoding='utf8')

In [60]:
album_df6 = pd.read_csv('./album_df5.csv')

### 앨범 정보 불러오기

+ Mnet

In [93]:
album_info = [[] for i in range(5+1)]

for i in range(len(album_df6)):
    url = 'http://www.mnet.com/album/{}'.format(album_df6.loc[i,'Mnet'])
    response = requests.get(url) 
    soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

    album_num = soup.select('div.a_info_cont > dl > dd > p > span')
    

    if len(album_num)==12:
        [album_info[j].append(info.text) for j, info in enumerate(album_num[1:len(album_num):2])]
    else:
        if album_num[-2].text == '기획사':
            [album_info[j].append(info.text) for j, info in enumerate(album_num[1:len(album_num):2])]
            [album_info[j].append(None) for j in list(range(len(album_num)//2,6))]
        else :
            [album_info[j].append(info.text) for j, info in enumerate(album_num[1:8:2])]
            album_info[4].append(None)
            album_info[5].append(album_num[-1].text)

In [94]:
for i, column in enumerate(['singer', 'Release_Date', 'Album_type', 'Music_genre', 'Agency', 'Distributor']):
    album_df6[column] = album_info[i]

In [96]:
album_df6.to_csv('album_df6.csv', index=False, encoding='utf8')

In [144]:
album_df7 = pd.read_csv('./album_df6.csv')

# 댓글을 가져오자!

+ 네이버

In [ ]:
chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver)

comments_list = []
i = 0
url = 'http://music.naver.com/album/index.nhn?albumId={}'.format(album_df6.loc[i,'naver'])
driver.get(url)

for page in range(2,5):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comments = soup.select('#comment_focus > li > div > div > p')
    for comment in comments:
        comments_list.append(comment.text)
    next_selector = '#cbox_module > div.cbox_paginate > a.N\3d a\3a CML\2e page\2c r\3a {}'.format(page)
    next_Button = driver.find_element_by_css_selector(next_selector)   
    next_Button.click()  

time.sleep(3)
driver.quit()

+ Mnet

In [ ]:
for page in range(2,5):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comments = soup.select('#comment_focus > li > div > div > p')
    for comment in comments:
        comments_list.append(comment.text)
    next_selector = '#cbox_module > div.cbox_paginate > a.N\3d a\3a CML\2e page\2c r\3a {}'.format(page)
    next_Button = driver.find_element_by_css_selector(next_selector)   
    next_Button.click()  

time.sleep(3)
driver.quit()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver)

i = 0
url = 'http://music.naver.com/album/index.nhn?albumId={}'.format(album_df6.loc[i,'naver'])
driver.get(url)

count = 0
try:
    element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.ID, "moreviewDIV"))
    )
except TimeoutException:
    print("관련 댓글이 더 없습니다.")
else:
    loop = True
    while loop and count < 1:
        try:
            element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#moreviewDIV"))
            )
            more_button = driver.find_element_by_css_selector("#moreviewDIV > a")
            webdriver.ActionChains(driver).move_to_element(more_button).click(more_button).perform() 
            count += 1
        except: 
            loop = False
            
#     title = driver.find_element_by_css_selector("#cSub > div.head_view > h3")
#     print('[',title.text, ']\n')
#     comment = driver.find_element_by_css_selector("#alex-area > div > div > div > div.cmt_box > ul.list_comment")
#     comment_list = comment.find_elements_by_tag_name('p')
#     for num, p in enumerate(comment_list):
#         print(num+1, ']', p.text, "\n\n")
#         if num == 9:
#             break
    print(a)
    break        
driver.quit()

+ melon

In [310]:
def mul_replace(comment):
    a = re.sub(r'내용\s', '', comment.text)
    a = re.sub(r'\s*재생 다운로드.*\s*', '', a)
    a = re.sub(r'\s', '', a)
    return a

In [311]:
chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver)

melon_commend_list = [[] for i in range(len(album_df7))]

for i in range(len(album_df7)):
    url = 'http://www.melon.com/album/detail.htm?albumId={}'.format(album_df7.loc[i,'melon'])
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    count = soup.select('#d_cmtpgn_cmt_count_wrapper > div > strong > span')
    comments = soup.select('#d_cmtpgn_cmt_list_wrapper > ul > li > div > div.wrap_cntt.d_cmtpgn_cmt_cont_wrapper > div.cntt > div')
    
    for com in comments:
        melon_commend_list[i].append(mul_replace(com))
        
    for j in range(2, int(re.sub(',', '', count[0].text))//10+1):
        url = 'http://www.melon.com/album/detail.htm?albumId={}#cmtpgn=&pageNo={}&sortType=0&srchType=2&srchWord='.format(album_df7.loc[i,'melon'], j)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        count = soup.select('#d_cmtpgn_cmt_count_wrapper > div > strong > span')
        comments = soup.select('#d_cmtpgn_cmt_list_wrapper > ul > li > div > div.wrap_cntt.d_cmtpgn_cmt_cont_wrapper > div.cntt > div')

#         for com in comments:
#             melon_commend_list.append(mul_replace(com))
        for com in comments:
            melon_commend_list[i].append(mul_replace(com))

driver.quit()

TimeoutException: Message: timeout
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.32.498550 (9dec58e66c31bcc53a9ce3c7226f0c1c5810906a),platform=Windows NT 10.0.16299 x86_64)


In [ ]:
http://www.melon.com/album/detail.htm?albumId=10008861#cmtpgn=&pageNo=6476&sortType=0&srchType=2&srchWord=

In [312]:
album_df7['melon_commend'] = melon_commend_list

In [322]:
album_df7.to_csv('album_df7.csv', index=False, encoding='utf8')